In [1]:
import pandas as pd
import numpy as np

In [2]:
data_set = pd.read_csv('./data/Womens Clothing E-Commerce Reviews.csv')
data_set = data_set.drop(data_set.columns[0], axis=1)

In [3]:
reviews = data_set['Review Text'].sample(n=100)

rr = reviews.values
rr = rr.tolist()
rr = list(map(lambda x: x if type(x) == str else "", rr))

a = [10*n for n in range(11)]

bb = [rr[a[n-1]:a[n]] for n, _ in enumerate(a)][1:]

for n in range(10):
    file = open('./Review/file%s.txt'%n, 'w')
    file.write("\n".join(bb[n]))
    file.close()

In [4]:
bins_age = pd.IntervalIndex.from_tuples([(0, 25), (25, 60), (60, 200)])
data_set['age_category'] = pd.cut(data_set.Age, bins_age)
data_set.age_category.cat.categories = ["jovens", "adultos", "terceira idade"]

In [5]:
bins_rating = pd.IntervalIndex.from_tuples([(0, 2), (2, 4), (4, 10)])
data_set['rating_category'] = pd.cut(data_set.Rating, bins_rating)
data_set.rating_category.cat.categories = ["muito ruim", "razoável", "boa"]

In [15]:
division_name = data_set['Division Name']
division_name.unique()

array(['Initmates', 'General', 'General Petite', nan], dtype=object)

In [16]:
departament_name = data_set['Department Name']
departament_name.unique()

array(['Intimate', 'Dresses', 'Bottoms', 'Tops', 'Jackets', 'Trend', nan],
      dtype=object)

In [23]:
class_name = data_set['Class Name']
class_name.unique()

array(['Intimates', 'Dresses', 'Pants', 'Blouses', 'Knits', 'Outerwear',
       'Lounge', 'Sweaters', 'Skirts', 'Fine gauge', 'Sleep', 'Jackets',
       'Swim', 'Trend', 'Jeans', 'Legwear', 'Shorts', 'Layering',
       'Casual bottoms', nan, 'Chemises'], dtype=object)

In [4]:
print(data_set.isnull().sum())
data_set = data_set.dropna(subset=['Review Text'])
data_set = data_set.sample(n=1000)
data_set.to_csv('./data/only1000.csv', index=False)

Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64


In [5]:
!pip install requests 

    100% |████████████████████████████████| 61kB 442kB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 2.3MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/62/00/ee1d7de624db8ba7090d1226aebefab96a2c71cd5cfa7629d6ad3f61b79e/urllib3-1.24.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 153kB 1.7MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 5.9MB/s ta 0:00:01
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
import requests as rq
import json

def analyze(text):
    payload = {'text': text}
    headers = {'content-type': 'application/json'}

    gl = rq.post('http://localhost:3000/analyze', json=payload, headers=headers)
    return gl.content

In [21]:
ds = data_set.sample(n=3)

In [23]:
ds["result"] = ds['Review Text'].apply(lambda x: analyze(x))
ds

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,result
5313,895,54,Keeper,Wasn't expecting this to look good on when i g...,4,1,0,General,Tops,Fine gauge,"b'{""usage"":{""text_units"":1,""text_characters"":5..."
12687,866,50,NaN,Such a cute top! can be dressed up or down. i ...,4,1,0,General,Tops,Knits,"b'{""usage"":{""text_units"":1,""text_characters"":9..."
19739,877,29,Too short and too wide,"I&#39;m taller (5&#39;8), so it&#39;s not unco...",2,0,0,General,Tops,Knits,"b'{""usage"":{""text_units"":1,""text_characters"":2..."


In [65]:
aa = ds['result'].apply(lambda x: json.loads(x)['entities'])

In [57]:
bb = aa.values.tolist()[2]

In [51]:
def bemMalNone(cases, cat='PRODUCT'):
    bem = list(filter(lambda x: x['disambiguation']['subtype'][0] == 'BEM', cases))
    mal = list(filter(lambda x: x['disambiguation']['subtype'][0] == 'MAL', cases))
    if len(bem) == 0 and len(mal) == 0:
        return (cat,'NONE')
    if len(bem) > len(mal):
        return (cat,'BEM')
    return (cat,'MAL')

def classify(res):
    prod = list(filter(lambda x: x['type'] == 'PRODUCT', res))
    comp = list(filter(lambda x: x['type'] == 'COMPANY', res))
    if len(prod) == 0 and len(comp) == 0:
        return ('NONE', 'NONE')
    if len(prod) > len(comp):
        return bemMalNone(prod, 'PRODUCT')
    return bemMalNone(comp, 'COMPANY')


In [66]:
ds['classify'] = aa.apply(lambda x: classify(x))

In [69]:
ds['tipo'] = ds['classify'].apply(lambda x: x[0])
ds['bemmal'] = ds['classify'].apply(lambda x: x[1])

In [70]:
ds

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,result,classify,tipo,bemmal
5313,895,54,Keeper,Wasn't expecting this to look good on when i g...,4,1,0,General,Tops,Fine gauge,"b'{""usage"":{""text_units"":1,""text_characters"":5...","(PRODUCT, BEM)",PRODUCT,BEM
12687,866,50,NaN,Such a cute top! can be dressed up or down. i ...,4,1,0,General,Tops,Knits,"b'{""usage"":{""text_units"":1,""text_characters"":9...","(NONE, NONE)",NONE,NONE
19739,877,29,Too short and too wide,"I&#39;m taller (5&#39;8), so it&#39;s not unco...",2,0,0,General,Tops,Knits,"b'{""usage"":{""text_units"":1,""text_characters"":2...","(COMPANY, NONE)",COMPANY,NONE
